#Sample Project Code
Assuming all other components were completed, this project code would earn 10/15 points for the code submission portion of the project because it is not very complex.

## setup libraries

In [ ]:
#library and code setup
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

!pip install gdelt

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad

In [ ]:
#start spark local server
import sys, os
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

## fetch gdelt data for a given date range

Note: this example only fetches the events table. The gdelt python API also can pull from the mentions and gkg tables. See https://linwoodc3.github.io/gdeltPyR/ for full API details



In [ ]:
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import gdelt
import os

# set up gdeltpyr for version 2
gd = gdelt.gdelt(version=2)

# multiprocess the query
e = ProcessPoolExecutor()


# generic functions to pull and write data to disk based on date
def get_filename(x):
  date = x.strftime('%Y%m%d')
  return "{}_gdeltdata.csv".format(date)

def intofile(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split("_")[0]
          d = gd.Search(date, table='events',coverage=False) #not updata at 15mins
          d.to_csv(filename,encoding='utf-8',index=False)
    except:
        print("Error occurred")

# pull the data from gdelt into multi files; this may take a long time
dates_before = [get_filename(x) for x in pd.date_range('2016 Nov 1','2016 Nov 7')]
dates_after = [get_filename(x) for x in pd.date_range('2016 Nov 9','2016 Nov 15')]

results = list(e.map(intofile,dates_before+dates_after))

/usr/local/lib/python3.7/dist-packages/gdelt/base.py:617: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = eventWork(self.download_list)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:617: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = eventWork(self.download_list)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:617: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = eventWork(self.download_list)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:617: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = eventWork(self.download_list)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:617: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = 

## read downloaded files into RDDs

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

data_before = sqlContext.read.option("header", "true").csv(dates_before)
data_after = sqlContext.read.option("header", "true").csv(dates_after)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
# Event codes categorised into positive, neutral, and negative events.
neutral_events = ['011', '019', '020', '024', '0241', '0242', '025', '0253', 
                  '0341', '0342', '0343', '0344', '035', '040', '041', '042',
                  '043', '044', '045', '046', '080', '083', '0831', '0832', '0833',
                  '0834', '084', '0841', '0842', '090', '091', '092', '093',
                  '094', '100', '104', '1041', '1042', '1043', '1044', '105', 
                  '106', '107', '108', '110', '123', '1231', '1232', '1233', 
                  '1234', '124', '125', '126', '127', '128', '129', '140', 
                  '141', '1411', '1412', '1413', '1414', '150', '160', '166', '170']


negative_events = ['012', '016', '111', '112', '1121', '1122', '1123', '1124',
                   '1125', '113', '115', '116', '120', '121', '1211', '1212', 
                   '122', '1221', '1222', '1223', '1224', '1241', '1242',
                   '1243', '1244', '1245', '1246', '130', '131', '1311', 
                   '1312', '1313', '132', '1321', '1322', '1323', '1324', 
                   '133', '134', '135', '136', '137', '1381', '138114', 
                   '1382', '1383', '1384', '1385', '139', '142', '1421', 
                   '1422', '1423', '1424', '143', '1431', '1432', '1433', 
                   '1434', '144', '1441', '1442', '1443', '1444', '145', 
                   '1451', '1452', '1453', '1454', '161', '162', '1621', 
                   '1622', '1623', '163', '164', '165', '1661', '1662', 
                   '1663', '171', '1711', '1712', '172', '1721', '1722', 
                   '1723', '1724', '173', '174', '175', '176', '180', '181', 
                   '182', '1821', '1822', '1823', '183', '1831', '1832', 
                   '1833', '1834', '184', '185', '186', '190195', '191', 
                   '192', '193', '194', '1951', '1952', '196', '200', '201',
                   '202', '203', '204', '2041', '2042']


positive_events = ['013', '014', '015', '017', '018', '021', '0211', '0212',
                   '0213', '0214', '022', '022', '023', '0231', '0232', '0233',
                   '0234', '0243', '0244', '0251', '0252', '0254', '0255', 
                   '0256', '026', '027', '028', '030', '031', '0311', '0312',
                   '0313', '0314', '032', '032', '033', '0331', '0332', '0333', 
                   '0334', '034', '0351', '0352', '0353', '0354', '0355', '0356', 
                   '036', '037', '038', '039', '050', '051', '052', '053', 
                   '054', '055', '056', '057', '060', '061', '062', '063',
                   '064', '070', '071', '072', '073', '074', '075', '081', 
                   '0811', '0812', '0813', '0814', '082', '085', '086', '0861',
                   '0862', '0863', '087', '0871', '0872', '0873', '0874',
                   '101', '1011', '1012', '1013', '1014', '102', '103', '1031',
                   '1032', '1033', '1034', '1051', '1052', '1053', '1054', 
                   '1055', '1056', '150', '151', '152', '153', '154', '155']

def event_sign(event_code):
  return ("Positive" if event_code in positive_events else
          "Neutral" if event_code in neutral_events else
          "Negative" if event_code in negative_events else "Unknown")
  




## Actual project code to answer a "research question":
How similar was the response of the US to the US election of 2016 to the responses of other countries to the US election of 2016?
Specifically, how is the response measured as an affect (positive/negative/etc) vector of event counts different between the countries?

In [ ]:
import numpy as np

def get_country_pos_neg_summary(data, countries):
  country_event_types = data.rdd.map(lambda row: ((row['Actor1CountryCode'], event_sign(row['EventCode'])), 1))
  country_event_types = country_event_types.filter(lambda x: x[0][0] is not None)
  #dbg(country_event_types)

  summary = country_event_types.reduceByKey(lambda a, b: a+b)
  country_summary = summary.filter(lambda kv: kv[0][0] in countries)
  return {k:v for k,v in country_summary.collect()}

def get_vector(result, country):
  return np.array([result[(country, 'Positive')],
          result[(country, 'Negative')],
          result[(country, 'Neutral')],
          result[(country, 'Unknown')]])

def cos_similarity(v1, v2):
  return v1.dot(v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))

test_countries = ['USA', 'CAN', 'MEX', 'GBR', 'DEU', 'NZL', 'CHN', 'RUS']

summary_before = get_country_pos_neg_summary(data_before, test_countries)
summary_after = get_country_pos_neg_summary(data_after, test_countries)

before = {c:get_vector(summary_before, c) for c in test_countries}
after = {c:get_vector(summary_after, c) for c in test_countries}

for c in test_countries:
  print(c, "vectors before->after", before[c], "->", after[c])


print("Similarity to US in attitude before and after US Election of 2016:")

reaction_similarity = []
for c in test_countries:
  before_similarity = cos_similarity(before['USA'], before[c])
  after_similarity = cos_similarity(after['USA'], after[c])
  print(c, "similiarity before->after", before_similarity, "->", after_similarity)
  reaction_similarity.append((after_similarity-before_similarity,c))

print(sorted(reaction_similarity))


USA vectors before->after [ 866  518 1098  344] -> [ 941  650 1193  397]
CAN vectors before->after [ 82  36 106  42] -> [57 34 93 33]
MEX vectors before->after [20  6  9  3] -> [ 8 32 25  4]
GBR vectors before->after [62 46 81 31] -> [69 31 79 27]
DEU vectors before->after [22 30 20  5] -> [13 16 42 17]
NZL vectors before->after [25  6 33  3] -> [17  5 15  7]
CHN vectors before->after [64 27 70 13] -> [58 26 37  9]
RUS vectors before->after [57 31 65 16] -> [48 26 49 19]
Similarity to US in attitude before and after US Election of 2016:
USA similiarity before->after 1.0 -> 1.0
CAN similiarity before->after 0.9937348415981692 -> 0.9884046630225544
MEX similiarity before->after 0.8929352991832146 -> 0.8453047746093852
GBR similiarity before->after 0.9967391194418906 -> 0.9924982439114011
DEU similiarity before->after 0.895213861426214 -> 0.9394837000534765
NZL similiarity before->after 0.9663022898822903 -> 0.9686147525109468
CHN similiarity before->after 0.9902263471780297 -> 0.94645346

In [ ]:
!zip -o mydata.zip *.csv

  adding: 20161101_gdeltdata.csv (deflated 83%)
  adding: 20161102_gdeltdata.csv (deflated 82%)
  adding: 20161103_gdeltdata.csv (deflated 83%)
  adding: 20161104_gdeltdata.csv (deflated 83%)
  adding: 20161105_gdeltdata.csv (deflated 83%)
  adding: 20161106_gdeltdata.csv (deflated 83%)
  adding: 20161107_gdeltdata.csv (deflated 83%)
  adding: 20161109_gdeltdata.csv (deflated 82%)
  adding: 20161110_gdeltdata.csv (deflated 83%)
  adding: 20161111_gdeltdata.csv (deflated 82%)
  adding: 20161112_gdeltdata.csv (deflated 85%)
  adding: 20161113_gdeltdata.csv (deflated 83%)
  adding: 20161114_gdeltdata.csv (deflated 82%)
  adding: 20161115_gdeltdata.csv (deflated 83%)
